In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler,LabelEncoder

# Load the Dataset Wine

In [ ]:
wine = pd.read_csv("Datasets/winequality/winequality-red.csv", sep=';')

In [ ]:
wine.head()

# Data Preorcessing 

In [ ]:
bins = (2, 6.5, 8)
group_names = ["bad", "good"]
wine["quality"] = pd.cut(wine["quality"], bins = bins, labels = group_names)
wine["quality"].unique()
label_quality = LabelEncoder()

In [ ]:
wine["quality"] = label_quality.fit_transform(wine["quality"])

In [ ]:
wine.head()

In [ ]:
wine["quality"].value_counts()

In [ ]:
X = wine.drop("quality", axis=1)
y = wine["quality"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# MLP Classifier

In [ ]:
# Stets some parameters for looking for the best parameters
modes = ["identity","logistic","tanh","relu"]
alphas = [1e-5,1e-4,1e-3]
solv = ["adam","sgd","lbfgs"]
#h = [10,100,200,400,500,800,1000]
h = [10,100]

In [ ]:
#alphas = [0.01]
#erg = []
#for l in h:
#    for i in alphas:
#        for j in modes:
#            for k in solv:
#                clf = MLPClassifier(max_iter=500,activation=j,hidden_layer_sizes=(l,l,l))
#                #clf = MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=300, alpha=i,solver='sgd', verbose=10,  random_state=21,tol=0.000000001,activation=j)
#                clf.fit(X_train, y_train)
#                Y_pred = clf.predict(X_test)
#                print("hidden_size: ",l,"alpha: ",i,"mode: ",j,"solver: ",k,"score: ",accuracy_score(y_test, Y_pred))
#                erg.append("hidden_size:")
#                erg.append(h)
#                erg.append("alpha:")
#                erg.append(i)
#                erg.append("mode:")
#                erg.append(j)
#                erg.append("solver:")
#                erg.append(k)
#                erg.append("score:")
#                erg.append(accuracy_score(y_test, Y_pred))
#            print("")
#        print("")
#    print("")
#print("")

In [ ]:
l = 500
clf = MLPClassifier(max_iter=500,activation="relu",hidden_layer_sizes=(l,l,l),alpha=0.01)
clf.fit(X_train, y_train)
Y_pred = clf.predict(X_test)
print("hidden_size: ",l,"score: ",accuracy_score(y_test, Y_pred))

## Statistics

In [ ]:
sns.distplot(y_test)
sns.distplot(Y_pred, color="red")

In [ ]:
sns.distplot(y_test-Y_pred)

# knn Classifier

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

print(sqrt(mean_squared_error(y_test, Y_pred)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
error_rate = []

for i in range(1,51):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i)) # maybe pointless?

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(range(1,51), error_rate, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)
plt.title('Error rate vs k-Value')
plt.xlabel('k-Value')
plt.ylabel('Error rate')

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors': range(1, 51)}

knn = KNeighborsClassifier()

model = GridSearchCV(knn, params, cv=10)
model.fit(X_train, y_train)
print("Best k-Value is: ", model.best_params_['n_neighbors'])

In [ ]:
pred = model.predict(X_test)

In [ ]:
solution_data = list(zip(list(range(750,1500)), pred))

In [ ]:
solution_table = pd.DataFrame(solution_data, columns=['ID', 'quality'])
solution_table.head()

# Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=900)
rfc.fit(X_train,y_train)
pred_rfc = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test,pred_rfc))
print(confusion_matrix(y_test, pred_rfc))

In [ ]:
sns.distplot(y_test)
sns.distplot(pred_rfc, color="red")

In [ ]:
sns.distplot(y_test-pred_rfc)

In [ ]:
print(sqrt(mean_squared_error(y_test, pred_rfc)))